In [1]:
from nba_api.stats.endpoints import teamgamelog, boxscoresummaryv2
import csv, json, time
import pandas as pd

In [2]:
teams = pd.read_csv("master_team_stats_2024.csv")
players = pd.read_csv("player_stats_2024.csv")
teams

,TEAM_ID,TEAM_NAME,W,L,W_PCT,FGM_RANK,FG3M_RANK,PLUS_MINUS_RANK,REB_RANK,AST_RANK,...,STL_RANK,BLK_RANK,PF_RANK,PTS_RANK,DivisionRank,HOME_WINS,ROAD_WINS,CurrentHomeStreak,CurrentRoadStreak,CurrentStreak
0,1610612737,Atlanta Hawks,40,42,0.488,6,14,18,14,2,...,2,12,22,5,2,21,19,2,2,3
1,1610612738,Boston Celtics,61,21,0.744,16,1,3,8,16,...,29,5,2,8,1,28,33,4,-1,2
2,1610612751,Brooklyn Nets,26,56,0.317,30,13,26,29,24,...,20,26,28,29,4,12,14,-2,-1,-3
3,1610612766,Charlotte Hornets,19,63,0.232,28,18,27,9,26,...,26,21,16,30,4,12,7,-3,-9,-7
4,1610612741,Chicago Bulls,39,43,0.476,8,3,20,3,5,...,24,18,6,6,5,18,21,4,1,3
5,1610612739,Cleveland Cavaliers,64,18,0.780,4,2,2,6,9,...,14,27,11,1,1,34,30,-1,1,-1
6,1610612742,Dallas Mavericks,39,43,0.476,13,26,19,23,22,...,21,7,8,15,3,22,17,1,-3,-1
7,1610612743,Denver Nuggets,50,32,0.610,1,28,9,4,1,...,17,15,6,3,2,26,24,1,2,3
8,1610612765,Detroit Pistons,44,38,0.537,10,20,13,13,15,...,19,9,27,12,4,22,22,-1,-1,-2
9,1610612744,Golden State Warriors,48,34,0.585,21,4,10,7,4,...,4,17,23,17,3,24,24,-3,6,-1


In [3]:
def get_last_3_games(season, season_type,team_id):
    return teamgamelog.TeamGameLog(season=season, season_type_all_star=season_type, team_id=team_id).get_data_frames()[0][0:3]
season = "2024-25"
get_last_3_games(season,"Regular Season", "1610612750")

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612750,0022401195,"APR 13, 2025",MIN vs. UTA,W,49,33,0.598,240,46,...,0.688,17,37,54,23,9,9,13,16,116
1,1610612750,0022401179,"APR 11, 2025",MIN vs. BKN,W,48,33,0.593,240,42,...,0.759,7,41,48,27,5,5,12,15,117
2,1610612750,0022401170,"APR 10, 2025",MIN @ MEM,W,47,33,0.588,240,48,...,0.893,8,33,41,29,6,3,16,25,141


In [4]:
def get_inactive_players(season,season_type,team_id):
    inactive_players = set()
    last_3 = get_last_3_games(season,season_type,team_id)
    for game in last_3['Game_ID']:
        summary = boxscoresummaryv2.BoxScoreSummaryV2(game)
        inactive_df = summary.inactive_players.get_data_frame()
        for player in inactive_df['PLAYER_ID']:
            player_team = players.loc[players['PLAYER_ID']==player, 'TEAM_ID'].squeeze()
            if str(player_team) == team_id:
                inactive_players.add(player)
        time.sleep(0.6)
    return inactive_players


In [12]:
inactive_team = get_inactive_players(season, "Regular Season", "1610612756")
for player in inactive_team:
    if int(players.loc[players['PLAYER_ID']==player,'PER'].squeeze())>25.0:
        print(players[players['PLAYER_ID']==player]['FULL_NAME'])

Player 201142 has PER over 25.0!
Player 1628983 has PER over 25.0!
135    Kevin Durant
Name: FULL_NAME, dtype: object


In [5]:
def injured_all_star(season, season_type,team_id):
    inactive_team_set = get_inactive_players(season, season_type,team_id)
    for player in inactive_team_set:
        if int(players.loc[players['PLAYER_ID']==player,'PER'].squeeze())>25 and int(players.loc[players['PLAYER_ID']==player,'MIN'].squeeze())>750:
            return True
    return False 

In [12]:
injured_all_star("2024-25","Regular Season","1610612738")

True

In [17]:
def set_team_injured(season,season_type):

    for team in teams['TEAM_ID']:
        injured_bool = injured_all_star(season,season_type,str(team))
        print(f"{teams.loc[teams['TEAM_ID']==team,'TEAM_NAME'].squeeze()}: {injured_bool}")


In [18]:
set_team_injured("2024-25","Regular Season")

Atlanta Hawks: False
Boston Celtics: True
Brooklyn Nets: False
Charlotte Hornets: True
Chicago Bulls: False
Cleveland Cavaliers: False
Dallas Mavericks: False
Denver Nuggets: False
Detroit Pistons: False
Golden State Warriors: False
Houston Rockets: True
Indiana Pacers: False
LA Clippers: False
Los Angeles Lakers: False
Memphis Grizzlies: False
Miami Heat: False
Milwaukee Bucks: False
Minnesota Timberwolves: False
New Orleans Pelicans: False
New York Knicks: False
Oklahoma City Thunder: True
Orlando Magic: False
Philadelphia 76ers: False
Phoenix Suns: True
Portland Trail Blazers: False
Sacramento Kings: False
San Antonio Spurs: True
Toronto Raptors: False
Utah Jazz: False
Washington Wizards: False
